In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import pprint

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter

import DecisionTree

%reload_ext autoreload
%autoreload 2
%matplotlib inline

KeyError: 5.1

In [4]:
# 书上题目5.1
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    # 返回数据集和每个维度的名称
    return datasets, labels

datasets, labels = create_data()
data_df = pd.DataFrame(datasets, columns=labels)

In [6]:
dt = DecisionTree.DecisionTree()
tree = dt.fit(data_df)
print(tree)
pred = dt.predict(['老年', '否', '否', '一般'])
print(pred)

{'label:': None, 'feature_name': '有自己的房子', 'child': {'否': {'label:': None, 'feature_name': '有工作', 'child': {'否': {'label:': '否', 'feature_name': None, 'child': {}}, '是': {'label:': '是', 'feature_name': None, 'child': {}}}}, '是': {'label:': '是', 'feature_name': None, 'child': {}}}}
否


In [7]:
dt = DecisionTree.DecisionTree(type = 'c4.5')
tree = dt.fit(data_df)
print(tree)
pred = dt.predict(['老年', '否', '否', '一般'])
print(pred)

{'label:': None, 'feature_name': '有自己的房子', 'child': {'否': {'label:': None, 'feature_name': '有工作', 'child': {'否': {'label:': '否', 'feature_name': None, 'child': {}}, '是': {'label:': '是', 'feature_name': None, 'child': {}}}}, '是': {'label:': '是', 'feature_name': None, 'child': {}}}}
否


In [15]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['label'] = iris.target
df.columns = [
    'sepal length', 'sepal width', 'petal length', 'petal width', 'label'
]

df2 = df.iloc[:100, [0, 1, -1]]
df_train, df_test = train_test_split(df2, test_size=0.3, random_state=999)

data = np.array(df2)
X, y = data[:, :2], data[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=999)

In [16]:
# scikit-learn实例
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

0.9666666666666667

In [18]:
dt = DecisionTree.DecisionTree()
tree = dt.fit(df_train)
dt.predict(df_test)

KeyError: 5.1

In [2]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.random.randn(10, 4), columns=list('ABCD'))
df

,A,B,C,D
0,0.471655,-0.816998,0.193365,-0.607563
1,-0.736051,1.557965,-0.805537,-1.077803
2,0.326934,0.478153,0.428207,1.331234
3,-0.890455,-1.543230,0.073069,0.011625
4,-0.373252,-0.801917,1.562971,-1.240917
5,-0.502169,1.073150,-1.871739,-0.315449
6,0.408367,1.075695,0.873187,1.013164
7,0.615050,-0.668378,-0.332954,-3.151282
8,-0.015128,-0.421552,0.759492,-0.089475
9,-0.878950,-0.235288,0.044126,-0.636301


In [8]:
for index, row in df.iterrows():
    row.loc['B']

-0.8169979633885219

1.5579653252505485

0.4781527889989276

-1.5432299644622616

-0.8019167266763083

1.0731500376593053

1.0756949927821422

-0.6683776162350991

-0.42155231737087695

-0.2352876697999919

In [9]:
from sklearn.datasets import load_iris
iris = load_iris()
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  